<a href="https://colab.research.google.com/github/lokeshsai-m/MTP/blob/main/Finetuning_textsummrization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('/content/dataset1.csv',encoding = 'unicode_escape')
data.drop_duplicates(keep='first')
# data.info()
# print(data.columns)
df = data[['ï»¿abstract', 'conclusion', 'tip', 'no']]

In [3]:
df = df.rename(columns={'no':'id', 'ï»¿abstract':'document', 'tip':'summary'})
# df = df.dropna()

In [4]:
df['document'] = df['document'].apply(lambda x: x.lower())
df['summary'] = df['summary'].apply(lambda x: x.lower())

In [5]:
!pip install transformers==4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 32.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [6]:
!pip install datasets rouge-score nltk -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00


In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
!pip install torch

In [9]:
import torch
import datasets
from datasets import Dataset
from datasets import load_metric

In [10]:
import transformers
print(transformers.__version__)

4.17.0


In [11]:
from transformers import AutoTokenizer

model_checkpoint ='t5-small'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

pad_on_right = tokenizer.padding_side == "right"

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [12]:
max_input_length = 1024
max_target_length = 128

In [13]:
df = df.sample(frac=1).reset_index(drop=True)
train = df[:40]
valid = df[41:46]
# test = df[47:]
# train.shape, valid.shape,test.shape

In [14]:
def preprocess_function(examples):
    inputs = ['summarize:' + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
import torch
import datasets
from datasets import Dataset

train = Dataset.from_pandas(train)
valid = Dataset.from_pandas(valid)

In [16]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_valid = valid.map(preprocess_function, batched=True)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [17]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [18]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [19]:
batch_size = 16
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [21]:
import gc
gc.collect()

275

In [22]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("[INFO] training using {}".format(torch.cuda.get_device_name(0)))

[INFO] training using Tesla T4


In [23]:
torch.cuda.empty_cache()

In [24]:
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


In [25]:
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-tips",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=200,
    predict_with_generate=True,
    fp16=True
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [26]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


In [27]:
metric = load_metric("rouge")

<ipython-input-27-7154407f1f07>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [28]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: conclusion, id, document, summary. If conclusion, id, document, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40
  Num Epochs = 200
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 600


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.705464,14.159700,3.211800,14.159700,14.249800,19.000000
2,No log,2.684461,13.031900,3.261100,12.955400,13.122000,19.000000
3,No log,2.659003,13.031900,3.261100,12.955400,13.122000,19.000000
4,No log,2.636806,14.181800,3.261100,14.181800,14.394800,19.000000
5,No log,2.614140,14.181800,3.261100,14.181800,14.394800,19.000000
6,No log,2.592252,15.296000,3.211800,15.296000,15.386100,19.000000
7,No log,2.574953,15.296000,3.211800,15.296000,15.386100,19.000000
8,No log,2.558644,14.012600,3.211800,14.012600,14.102700,19.000000
9,No log,2.544139,14.012600,3.211800,14.012600,14.102700,19.000000
10,No log,2.529220,14.012600,3.211800,14.012600,14.102700,19.000000


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: conclusion, id, document, summary. If conclusion, id, document, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: conclusion, id, document, summary. If conclusion, id, document, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: conclusion, id, document, summary. If conclusion, id, document, summary are not expected by `T5For

TrainOutput(global_step=600, training_loss=1.56685484568278, metrics={'train_runtime': 520.8909, 'train_samples_per_second': 15.358, 'train_steps_per_second': 1.152, 'total_flos': 2165468823552000.0, 'train_loss': 1.56685484568278, 'epoch': 200.0})

In [29]:
test = df[44:]

In [30]:
print(test.shape)

(5, 4)


In [31]:
eval_dataset = Dataset.from_pandas(test)

In [32]:
eval_dataset = eval_dataset.map(
                preprocess_function,
                batched=True)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [33]:
predict_results = trainer.predict(
            eval_dataset,max_length=128, num_beams=3)

The following columns in the test set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: conclusion, id, document, summary. If conclusion, id, document, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5
  Batch size = 16


In [34]:
metrics = predict_results.metrics

In [35]:
metrics

{'test_loss': 2.3210132122039795,
 'test_rouge1': 25.6882,
 'test_rouge2': 10.5882,
 'test_rougeL': 20.3535,
 'test_rougeLsum': 21.1573,
 'test_gen_len': 47.2,
 'test_runtime': 1.7359,
 'test_samples_per_second': 2.88,
 'test_steps_per_second': 0.576}

In [36]:
if args.predict_with_generate:
    predictions = tokenizer.batch_decode(predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    predictions = [pred.strip() for pred in predictions]

In [37]:
predictions

['a. marmelos has antidiarrhoeal, antimicrobial, antiviral, anticancer, chemopreventive, antipyretic, ulcer healing, antigenotoxic, diuretic, antifertility, and anti-inflammatory properties.',
 'cumin is an aromatic herb prepared from the dried seeds of the plantâ cuminum cyminumâ l. (family apiaceae) it is a major constituent of curry powder, and as a spice, its popularity is considered second only to pepper.',
 'raw red beetroot consumption affects metabolic markers and cognitive function in type 2 diabetes patients.',
 "germination improves the mung bean's nutritional value and its potential as a functional food.",
 'efy corms have antioxidant, antihelminthic, anticancer, antimicrobial, antidiabetic, anti-inflammatory, analgesic, gastroprotective, and hepatoprotective effects of bioactive chemicals present in efy corms.']

In [38]:
test['summary']

44    aegle marmelos (indian bael or bael fruit), ru...
45    cumin seeds are rich in protein, carbohydrates...
46    raw red beetroot consumption has positive impa...
47    mustard oil can contribute to the improvement ...
48    elephant foot yam is high in dietary fiber, mi...
Name: summary, dtype: object